In [152]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torch import nn
import torch.nn.functional as F
import os 
from torchvision import transforms
from tqdm import tqdm

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

path = os.getcwd()
train_data = torchvision.datasets.MNIST(path, download = True, train = True, transform = transforms.Compose([transforms.Pad(2), transforms.ToTensor()]))
test_data = torchvision.datasets.MNIST(path, download = True, train = False, transform = transforms.Compose([transforms.Pad(2), transforms.ToTensor()]))
train_loader = DataLoader(dataset=train_data, batch_size = 16, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size = 16, shuffle=True)

In [153]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.c1 = nn.Conv2d(in_channels = 1,out_channels = 6,kernel_size = 5, stride = 1, padding = 0)
        self.c2 = nn.Conv2d(in_channels = 6,out_channels = 16,kernel_size = 5, stride = 1, padding = 0)
        self.fc1 = nn.Linear(in_features = 16*5*5, out_features = 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 84)
        self.fc3 = nn.Linear(in_features = 84, out_features = 10)
    
    def forward(self,x):
        x = F.avg_pool2d(F.tanh(self.c1(x)), kernel_size = 2, stride = 2, padding = 0)
        x = F.avg_pool2d(F.tanh(self.c2(x)), kernel_size = 2, stride = 2, padding = 0)
        x = torch.flatten(x,1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x).float()
        x = F.log_softmax(x, dim = 1)
        # torch.argmax(x, dim = 1)
         
        return x 
        
leNet = LeNet()
print(leNet)
leNet(next(iter(train_loader))[0]/255).shape

LeNet(
  (c1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (c2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


torch.Size([16, 10])

In [ ]:
class network:
    def __init__(self,dataset,model,learning_rate = 1e-2):
        self.dataset = dataset
        self.model = model 
        self.lr = learning_rate
        self.loss_function = nn.CrossEntropyLoss()
        self.opt = torch.optim.SGD(self.model.parameters(), lr = self.lr)
        self.loss_collection = []
    
    def train(self, epochs):
        for epoch in tqdm(range(epochs)):
            for inputs, targets in self.dataset:
                outputs = self.model(inputs/255)
                
                loss = self.loss_function(outputs, targets.long())
                self.model.zero_grad()
                loss.backward()
                self.opt.step()
                
                self.loss_collection.append(loss)
                
n = network(train_loader,LeNet())
n.train(10)




















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...

In [151]:
import matplotlib.pyplot as plt 

acc = 0
for x, y in test_loader:
    output = torch.argmax(n.model(x), dim = 1)
    acc += sum([1 for i in range(len(y)) if output[i] ==  y[i]]) / len(y)
    
print(acc)


/Users/maciejbalawejder/opt/miniforge3/envs/tf24/lib/python3.8/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


71.125
